# Evolver Loop 3: Analysis of exp_003 Results

## Goal
Analyze the successful TF-IDF fixes from exp_003 and identify next steps to reach gold threshold (0.979080).

Current best: 0.6555 AUC (exp_003)
Gold threshold: 0.979080
Gap: 0.3236 points

## Key Findings from exp_003
- Character n-grams dominate feature importance (4 of top 10 features)
- Feature selection worked: 3,000 features better than 10,000
- Removed stop words to keep domain vocabulary
- CV improved from 0.6217 → 0.6555 (+0.0338)
- Low variance (±0.0104) indicates stable model

In [ ]:
import pandas as pd
import numpy as np
import json
import re
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from scipy.sparse import csr_matrix, hstack
import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Load data
print("Loading data...")
with open('/home/data/train.json', 'r') as f:
    train_data = json.load(f)
with open('/home/data/test.json', 'r') as f:
    test_data = json.load(f)

train = pd.DataFrame(train_data)
test = pd.DataFrame(test_data)

print(f"Train: {len(train)} samples, {sum(train['requester_received_pizza'])} positive ({sum(train['requester_received_pizza'])/len(train):.3f})")
print(f"Test: {len(test)} samples")

# Extract text
y = train['requester_received_pizza'].values
text_train = train['request_text_edit_aware'].fillna('').str.lower()
text_test = test['request_text_edit_aware'].fillna('').str.lower()